In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import wandb

entity = "andyrdt"
project = "othello_gpt_sae"

for artifact in [
    "sparse_autoencoder_othello-gpt_blocks.6.hook_resid_pre_512:latest",
    "sparse_autoencoder_othello-gpt_blocks.6.hook_resid_pre_1024:latest",
]:
    artifact_path = f"{entity}/{project}/{artifact}"
    api = wandb.Api()
    artifact = api.artifact(artifact_path)
    artifact.download()

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [14]:
from sae_training.utils import LMSparseAutoencoderSessionloader

path ="./artifacts/sparse_autoencoder_othello-gpt_blocks.6.hook_resid_pre_512:v0/final_sparse_autoencoder_othello-gpt_blocks.6.hook_resid_pre_512.pt"

model, sparse_autoencoder, activations_loader = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)

/opt/conda/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


LocalEntryNotFoundError: An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on.